<a href="https://colab.research.google.com/github/otanet/CognitiveBias_LT_20240623/blob/main/%E9%A1%9E%E4%BC%BC%E5%BA%A6_%E5%95%86%E5%93%81%E8%AA%AC%E6%98%8E%E6%96%87_20240729.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###・ 紅麹サプリメントの説明文とサントリーウエルネスの商品の説明文との類似度を計算し、リコメンドのアイテムとして、非類似度の商品をリコメンドすると、紅麹を連想させないですみます。     
###・類似度(または非類似度)の計算は、１）文脈を加味した大規模言語モデル(Transformer)、２）文脈を考慮した言語モデル(BERT)、３）テキストマイニングでの類似度をそれぞれ計算します。

### １）Transformer

In [ ]:
!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# モデルとトークナイザーの初期化
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# テキストをベクトルに変換する関数
def embed_text(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state[:, 0, :]  # ベクトルの取得
    return embeddings.cpu().numpy()

# 紅麹サプリメントの説明文
red_yeast_rice = "紅麹サプリメントは、関節の健康をサポートし、筋肉や軟骨の機能を改善します。コレステロール値の管理にも効果があります。"

# サントリーウエルネスのサプリメントのリスト（商品名、説明文）
supplements = [
    ("ロコモア", "趣味やスポーツに、旅行や新たな出会いに。ご自分の脚で歩き続けられる力は、毎日をもっと豊かなものにします。そして、その要となるのはやはり脚の健康です。そこで、これからも歩き続けたいと願う、すべての方にお応えしたいとの思いでサントリーは研究を重ねてきました。そして、〝脚の筋肉を力強くサポートする〞ことと、〝ひざ関節のスムーズさを支える〞ことが、歩くために重要であることをつきとめたのです。この着眼点をもとに、脚の筋肉の力となる筋肉成分とひざ関節のスムーズさを支える軟骨成分の今までにない組み合わせを実現したのが、歩行機能領域において4成分の組み合わせによる日本初※の機能性表示食品『ロコモア』です。"),
    ("グルコサミンアクティブ", "これからもアクティブに過ごすために、ひざ関節はとても大切です。そこでサントリーが着目したのがグルコサミンとコンドロイチンのダブル軟骨成分と、いたわり成分ケルセチンプラスでした。『グルコサミン アクティブ』はひざ関節サプリメントで日本初※4の、この３成分の組み合わせにより、移動時のひざ関節の悩みを改善することが報告されています。おかげさまでサントリーは連続売上No.1※1を達成。累計販売本数は3,000万本※2を突破しました。また満足度も約96％※3と多くのお客様に支持していただいております。より活動的に、自分の好きなことを楽しめる人生に『グルコサミン アクティブ』をお役立てください。"),
    ("セサミンEX", "セサミンEXは、若々しさを維持する力が減少していく、40代以降のからだのために作られたサプリメントです。ゴマ一粒に1%未満しか含まれない健康成分セサミンをはじめ、調子を整えるオリザプラス、年齢美容に 役立つ天然ビタミンE、トコトリエノールを配合。4つの厳選成分で、若々しい毎日へ。"),
    ("グルコサミン&コンドロイチン 顆粒", "粒を飲むのは苦手というお客様の声にお応えして生まれた顆粒タイプ。 “軟骨成分”コンドロイチンとグルコサミンに加え、健やかな歩みにうれしい「ケルセチンプラス」を配合。"),
    ("DHA&EPA＋セサミンEX", "５年後、10年後の健康を考えるのであれば、全身の健康の土台をしっかりと整える「基礎健康力」が大切です。私たちの体のすみずみに存在するDHAやEPAはその「基礎健康力」の鍵を握る成分。サントリー「DHA＆EPA+セサミンEX」は、食事だけでは摂ることが難しい青魚のサラサラ成分DHA・EPAをはじめ、ゴマの稀少成分セサミンなど、将来の健康維持を支える、日本食のエッセンスをヒントに厳選した8種の成分を一度に摂ることができるサプリメントです。200万人※1を超えるお客様にご愛飲いただき、累計販売本数は6,000万本※2を突破。DHAサプリメント市場18年連続売上No.1※3を獲得しています。目の前の健康対策、5年後、10年後の前向きで健康的な毎日のため、『DHA&EPA+セサミンEX』をぜひお役立てください。"),
    ("グラン マカ", "「活力ある毎日を送りたい」「いつまでもときめきを感じていたい」とは思うものの、年齢とともに気になってくる活力の低下。そこで、男として自信を取り戻して人生を楽しんでいただくために、おすすめしたいのがサントリーの『グラン マカ』です。活力を生むことで知られる「マカ」をはじめ、めぐりを促すスーパーアミノ酸「L-シトルリン」、さらに「亜鉛」など計6つの男を上げる成分が活力あふれる毎日をサポートします。"),
    ("オメガエイド", "「考える力に自信をもっていたい」「ずっと周りに頼られる存在でありたい」。いつまでも自分らしく、前向きな毎日を過ごすことは、多くの人に共通する願い。私たちサントリーは長年研究を重ね、脳の健康を支えるDHA、EPA、ARA(アラキドン酸)という3つのオメガ脂肪酸の組み合わせにたどり着きました。あなたの聡明で前向きな毎日を『オメガエイド』がしっかりとサポートします。"),
    ("イチョウ葉", "冴えのある毎日や、四季を通じて心地いい毎日。イチョウ葉はこれらと深く関わる“カラダのめぐり”をサポートする健康ハーブとして古くから研究されてきました。サントリーは、このイチョウ葉エキスをヨーロッパの摂取基準量1日120mg配合。更に、海のカロテノイド｢アスタキサンチン｣をプラスしたダブルパワーで、しっかり全身に送り届けます。"),
    ("グルコサミン&コンドロイチン 顆粒", "粒を飲むのは苦手というお客様の声にお応えして生まれた顆粒タイプ。 “軟骨成分”コンドロイチンとグルコサミンに加え、健やかな歩みにうれしい「ケルセチンプラス」を配合。こだわりのトリプルパワーで、笑顔で歩く毎日を力強く応援します。ほのかなレモン風味で、いつでも手軽にお飲みいただけます。"),
    ("黒酢にんにく", "若い頃との違いを感じ始めた中高年の方に。黒酢の名付け親でもある鹿児島県 坂元醸造の黒酢 と、青森県のブランド種にんにく 福地ホワイト六片 。これら2つの元気食材を1粒に凝縮したのが、『黒酢にんにく』。こだわりのダブルパワーで、体の中の元気をしっかりと支えます。"),
    ("極の青汁(きわみのあおじる)", "土作りからこだわった熊本県阿蘇※1など国産の「大麦若葉」、大自然の底力がみなぎる鹿児島県屋久島など国産の「明日葉」。この２つのこだわりの国産野菜の栄養を１杯に凝縮。さらに、野菜ポリフェノール「ケルセチン」の吸収力を高めた「ケルセチンプラス」を独自配合※2し、体にしっかり届けます。宇治抹茶と宇治煎茶をブレンドし、毎日飲みたくなる上品な味わいに仕上げました。"),
    ("青汁ゴーヤ", "青汁独特の風味が苦手という方にも大好評のサントリーの『青汁ゴーヤ』。多彩な栄養を含む国産の大麦若葉と、スタミナ野菜で知られる、沖縄産ゴーヤの、2つの“野菜力”を凝縮。さらにフラクトオリゴ糖と抹茶を加え、おいしく食べやすい、粒タイプの青汁に仕上げました。"),
]

# 各テキストをベクトルに変換
red_yeast_rice_embedding = embed_text(red_yeast_rice)
supplement_embeddings = [(name, embed_text(description)) for name, description in supplements]

# コサイン類似度の計算
similarities = [(name, cosine_similarity(red_yeast_rice_embedding, embed)[0][0]) for name, embed in supplement_embeddings]

# 結果を表示
sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
for name, sim in sorted_similarities:
    print(f"商品名: {name}, 類似度: {sim:.4f}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

商品名: セサミンEX, 類似度: 0.8512
商品名: グルコサミンアクティブ, 類似度: 0.8383
商品名: グルコサミン&コンドロイチン 顆粒, 類似度: 0.8083
商品名: 青汁ゴーヤ, 類似度: 0.8080
商品名: グルコサミン&コンドロイチン 顆粒, 類似度: 0.7978
商品名: ロコモア, 類似度: 0.7968
商品名: イチョウ葉, 類似度: 0.7899
商品名: オメガエイド, 類似度: 0.7773
商品名: 極の青汁(きわみのあおじる), 類似度: 0.7675
商品名: グラン マカ, 類似度: 0.7565
商品名: DHA&EPA＋セサミンEX, 類似度: 0.7409
商品名: 黒酢にんにく, 類似度: 0.7241


### BERT

In [ ]:
# 必要なライブラリのインストール
!pip install mecab-python3
!apt-get install -y mecab mecab-ipadic-utf8 libmecab-dev swig
!pip install unidic-lite
!python -m unidic.download

!pip install scikit-learn
!pip install numpy
!pip install pandas

import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from google.colab import files

# MeCabとunidicの初期化
import unidic_lite
mecab = MeCab.Tagger("-Owakati")

# ストップワードのファイルをアップロード
uploaded = files.upload()

# ストップワードの読み込み関数
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return set([line.strip() for line in f])

# ストップワードの設定（ユーザーがアップロードしたファイルを使用）
stop_words = load_stopwords('stopword_japanese.txt')

def preprocess_text(text):
    # テキストを分かち書きし、ストップワードを除去
    tokens = mecab.parse(text).split()
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# 紅麹サプリメントの説明文
red_yeast_rice = "紅麹サプリメントは、関節の健康をサポートし、筋肉や軟骨の機能を改善します。コレステロール値の管理にも効果があります。"

# サントリーウエルネスのサプリメントのリスト（商品名、説明文、クチコミ）
supplements = [
    ("ロコモア", "趣味やスポーツに、旅行や新たな出会いに。ご自分の脚で歩き続けられる力は、毎日をもっと豊かなものにします。そして、その要となるのはやはり脚の健康です。そこで、これからも歩き続けたいと願う、すべての方にお応えしたいとの思いでサントリーは研究を重ねてきました。そして、〝脚の筋肉を力強くサポートする〞ことと、〝ひざ関節のスムーズさを支える〞ことが、歩くために重要であることをつきとめたのです。この着眼点をもとに、脚の筋肉の力となる筋肉成分とひざ関節のスムーズさを支える軟骨成分の今までにない組み合わせを実現したのが、歩行機能領域において4成分の組み合わせによる日本初※の機能性表示食品『ロコモア』です。"),
    ("グルコサミンアクティブ", "これからもアクティブに過ごすために、ひざ関節はとても大切です。そこでサントリーが着目したのがグルコサミンとコンドロイチンのダブル軟骨成分と、いたわり成分ケルセチンプラスでした。『グルコサミン アクティブ』はひざ関節サプリメントで日本初※4の、この３成分の組み合わせにより、移動時のひざ関節の悩みを改善することが報告されています。おかげさまでサントリーは連続売上No.1※1を達成。累計販売本数は3,000万本※2を突破しました。また満足度も約96％※3と多くのお客様に支持していただいております。より活動的に、自分の好きなことを楽しめる人生に『グルコサミン アクティブ』をお役立てください。"),
    ("セサミンEX", "セサミンEXは、若々しさを維持する力が減少していく、40代以降のからだのために作られたサプリメントです。ゴマ一粒に1%未満しか含まれない健康成分セサミンをはじめ、調子を整えるオリザプラス、年齢美容に 役立つ天然ビタミンE、トコトリエノールを配合。4つの厳選成分で、若々しい毎日へ。"),
    ("グルコサミン&コンドロイチン 顆粒", "粒を飲むのは苦手というお客様の声にお応えして生まれた顆粒タイプ。 “軟骨成分”コンドロイチンとグルコサミンに加え、健やかな歩みにうれしい「ケルセチンプラス」を配合。"),
    ("DHA&EPA＋セサミンEX", "５年後、10年後の健康を考えるのであれば、全身の健康の土台をしっかりと整える「基礎健康力」が大切です。私たちの体のすみずみに存在するDHAやEPAはその「基礎健康力」の鍵を握る成分。サントリー「DHA＆EPA+セサミンEX」は、食事だけでは摂ることが難しい青魚のサラサラ成分DHA・EPAをはじめ、ゴマの稀少成分セサミンなど、将来の健康維持を支える、日本食のエッセンスをヒントに厳選した8種の成分を一度に摂ることができるサプリメントです。200万人※1を超えるお客様にご愛飲いただき、累計販売本数は6,000万本※2を突破。DHAサプリメント市場18年連続売上No.1※3を獲得しています。目の前の健康対策、5年後、10年後の前向きで健康的な毎日のため、『DHA&EPA+セサミンEX』をぜひお役立てください。"),
    ("グラン マカ", "「活力ある毎日を送りたい」「いつまでもときめきを感じていたい」とは思うものの、年齢とともに気になってくる活力の低下。そこで、男として自信を取り戻して人生を楽しんでいただくために、おすすめしたいのがサントリーの『グラン マカ』です。活力を生むことで知られる「マカ」をはじめ、めぐりを促すスーパーアミノ酸「L-シトルリン」、さらに「亜鉛」など計6つの男を上げる成分が活力あふれる毎日をサポートします。"),
    ("オメガエイド", "「考える力に自信をもっていたい」「ずっと周りに頼られる存在でありたい」。いつまでも自分らしく、前向きな毎日を過ごすことは、多くの人に共通する願い。私たちサントリーは長年研究を重ね、脳の健康を支えるDHA、EPA、ARA(アラキドン酸)という3つのオメガ脂肪酸の組み合わせにたどり着きました。あなたの聡明で前向きな毎日を『オメガエイド』がしっかりとサポートします。"),
    ("イチョウ葉", "冴えのある毎日や、四季を通じて心地いい毎日。イチョウ葉はこれらと深く関わる“カラダのめぐり”をサポートする健康ハーブとして古くから研究されてきました。サントリーは、このイチョウ葉エキスをヨーロッパの摂取基準量1日120mg配合。更に、海のカロテノイド｢アスタキサンチン｣をプラスしたダブルパワーで、しっかり全身に送り届けます。"),
    ("グルコサミン&コンドロイチン 顆粒", "粒を飲むのは苦手というお客様の声にお応えして生まれた顆粒タイプ。 “軟骨成分”コンドロイチンとグルコサミンに加え、健やかな歩みにうれしい「ケルセチンプラス」を配合。こだわりのトリプルパワーで、笑顔で歩く毎日を力強く応援します。ほのかなレモン風味で、いつでも手軽にお飲みいただけます。"),
    ("黒酢にんにく", "若い頃との違いを感じ始めた中高年の方に。黒酢の名付け親でもある鹿児島県 坂元醸造の黒酢 と、青森県のブランド種にんにく 福地ホワイト六片 。これら2つの元気食材を1粒に凝縮したのが、『黒酢にんにく』。こだわりのダブルパワーで、体の中の元気をしっかりと支えます。"),
    ("極の青汁(きわみのあおじる)", "土作りからこだわった熊本県阿蘇※1など国産の「大麦若葉」、大自然の底力がみなぎる鹿児島県屋久島など国産の「明日葉」。この２つのこだわりの国産野菜の栄養を１杯に凝縮。さらに、野菜ポリフェノール「ケルセチン」の吸収力を高めた「ケルセチンプラス」を独自配合※2し、体にしっかり届けます。宇治抹茶と宇治煎茶をブレンドし、毎日飲みたくなる上品な味わいに仕上げました。"),
    ("青汁ゴーヤ", "青汁独特の風味が苦手という方にも大好評のサントリーの『青汁ゴーヤ』。多彩な栄養を含む国産の大麦若葉と、スタミナ野菜で知られる、沖縄産ゴーヤの、2つの“野菜力”を凝縮。さらにフラクトオリゴ糖と抹茶を加え、おいしく食べやすい、粒タイプの青汁に仕上げました。"),
]

# テキストの前処理
processed_texts = [preprocess_text(red_yeast_rice)] + [preprocess_text(f"{desc} {review}") for _, desc, review in supplements]

# TF-IDFベクトル化
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_texts)

# コサイン類似度の計算
cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

# 結果をDataFrameに格納
results = pd.DataFrame({
    '商品名': [name for name, _, _ in supplements],
    '類似度': cosine_similarities,
    '商品説明': [desc for _, desc, _ in supplements],
    'クチコミ': [review for _, _, review in supplements]
})

# 類似度でソートし、上位10個と下位10個を選択（実際のアイテム数によって調整する必要があります）
top_10 = results.nlargest(10, '類似度')
bottom_10 = results.nsmallest(10, '類似度')

print("類似商品（上位10個）:")
print(top_10[['商品名', '類似度']])
print("\n非類似商品（下位10個）:")
print(bottom_10[['商品名', '類似度']])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 3.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-utils swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils swig swig4.0
0 upgraded, 8 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,483 kB of archives.
After this operation, 64.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab2 amd64 0.996-14build9 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab-dev amd64 0.996-14build9 [306 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 mecab-utils amd64 0.996-14build9 [4,850 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 m

Saving stopword_japanese.txt to stopword_japanese.txt
類似商品（上位10個）:
                  商品名       類似度
0                ロコモア  0.143373
1         グルコサミンアクティブ  0.102806
2         ひざサポートコラーゲン  0.074791
3           ヒアロモイスチャー  0.053122
4  ヘルシア 茶カテキンの力α 緑茶風味  0.052169

非類似商品（下位10個）:
                  商品名       類似度
4  ヘルシア 茶カテキンの力α 緑茶風味  0.052169
3           ヒアロモイスチャー  0.053122
2         ひざサポートコラーゲン  0.074791
1         グルコサミンアクティブ  0.102806
0                ロコモア  0.143373


### テキストマイニング

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# NLTKのデータをダウンロード
nltk.download('punkt')
nltk.download('stopwords')

# ストップワードの設定（日本語のストップワードリストを使用）
stop_words = set(stopwords.words('/content/stopword_japanese.txt'))
#stop_words = set(stopwords.words('japanese'))

def preprocess_text(text):
    # テキストを小文字に変換し、トークン化
    tokens = word_tokenize(text.lower())
    # ストップワードを除去
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# 紅麹サプリメントの説明文
red_yeast_rice = "紅麹サプリメントは、関節の健康をサポートし、筋肉や軟骨の機能を改善します。コレステロール値の管理にも効果があります。"

# サントリーウエルネスのサプリメントのリスト（商品名、説明文、クチコミ）
supplements = [
    ("ロコモア", "筋肉成分と軟骨成分で元気な脚へ。移動時のひざ関節の悩みを改善する成分を配合。", "膝の痛みが軽減され、動きやすくなった。毎日続けやすい。"),
    ("グルコサミンアクティブ", "グルコサミンとコンドロイチンを配合し、関節の健康をサポート。", "関節の痛みが和らいだ。飲みやすいカプセルタイプ。"),
    ("セサミンEX", "セサミンを配合し、抗酸化作用で健康をサポート。", "体調が良くなった。続けやすい。"),
    ("DHA&EPA＋セサミンEX", "DHAとEPA、セサミンを配合し、心血管の健康をサポート。", "血圧が下がった。飲みやすい。"),
    ("プロテクト乳酸菌", "乳酸菌を配合し、腸内環境をサポート。", "お腹の調子が良くなった。飲みやすい。"),
    ("マカ", "マカを配合し、スタミナと活力をサポート。", "疲れにくくなった。続けやすい。"),
    ("アラキドン酸(ARA)", "アラキドン酸を配合し、脳の健康をサポート。", "集中力が上がった。飲みやすい。"),
    ("グルコサミン&コンドロイチン", "グルコサミンとコンドロイチンを配合し、関節の健康をサポート。", "関節の動きが良くなった。続けやすい。"),
    ("黒酢にんにく", "黒酢とにんにくを配合し、体力とスタミナをサポート。", "疲れにくくなった。飲みやすい。"),
    ("野菜青汁", "野菜不足を補うための青汁。", "野菜不足が解消された。味が良い。"),
    ("ヒアロモイスチャー", "ヒアルロン酸を配合し、肌の保湿と潤いをサポート。", "肌がしっとりしてきた。乾燥が気にならなくなった。"),
    ("ヘルシア 茶カテキンの力α 緑茶風味", "茶カテキンを含み、脂肪の燃焼をサポートする飲料。", "体脂肪が減った。味が良くて続けやすい。"),
    ("コラーゲンC", "コラーゲンとビタミンCを配合し、美肌をサポート。", "肌がプルプルになった。飲みやすい。"),
    ("ビタミンD3", "ビタミンD3を配合し、骨の健康をサポート。", "骨密度が上がった。飲みやすい。"),
    ("オメガ3", "オメガ3脂肪酸を配合し、心血管の健康をサポート。", "血圧が下がった。続けやすい。"),
    ("プロバイオティクス", "乳酸菌を配合し、腸内環境をサポート。", "お腹の調子が良くなった。飲みやすい。"),
    ("鉄分サプリ", "鉄分を配合し、貧血予防をサポート。", "貧血が改善された。飲みやすい。"),
    ("マルチビタミン", "複数のビタミンを配合し、総合的な健康をサポート。", "体調が良くなった。続けやすい。"),
    ("カルシウム", "カルシウムを配合し、骨の健康をサポート。", "骨密度が上がった。飲みやすい。"),
    ("ビタミンC", "ビタミンCを配合し、美肌と免疫力をサポート。", "肌が明るくなった。風邪をひきにくくなった。"),
]

# テキストの前処理
processed_texts = [preprocess_text(red_yeast_rice)] + [preprocess_text(f"{desc} {review}") for _, desc, review in supplements]

# TF-IDFベクトル化
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_texts)

# コサイン類似度の計算
cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

# 結果をDataFrameに格納
results = pd.DataFrame({
    '商品名': [name for name, _, _ in supplements],
    '類似度': cosine_similarities,
    '商品説明': [desc for _, desc, _ in supplements],
    'クチコミ': [review for _, _, review in supplements]
})

# 類似度でソートし、上位10個と下位10個を選択
top_10 = results.nlargest(10, '類似度')
bottom_10 = results.nsmallest(10, '類似度')

print("類似商品（上位10個）:")
print(top_10[['商品名', '類似度']])
print("\n非類似商品（下位10個）:")
print(bottom_10[['商品名', '類似度']])

類似商品（上位10個）:
              商品名  類似度
0            ロコモア  0.0
1     グルコサミンアクティブ  0.0
2          セサミンEX  0.0
3  DHA&EPA＋セサミンEX  0.0
4        プロテクト乳酸菌  0.0
5              マカ  0.0
6     アラキドン酸(ARA)  0.0
7  グルコサミン&コンドロイチン  0.0
8          黒酢にんにく  0.0
9            野菜青汁  0.0

非類似商品（下位10個）:
              商品名  類似度
0            ロコモア  0.0
1     グルコサミンアクティブ  0.0
2          セサミンEX  0.0
3  DHA&EPA＋セサミンEX  0.0
4        プロテクト乳酸菌  0.0
5              マカ  0.0
6     アラキドン酸(ARA)  0.0
7  グルコサミン&コンドロイチン  0.0
8          黒酢にんにく  0.0
9            野菜青汁  0.0
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!